<a href="https://colab.research.google.com/github/aqhareus/floorplan-stable-diffusion-AI54/blob/main/AI54_Image_project_floorplan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install Dependencies (Corrected)

# 1. Install helper libraries first
!pip install -q transformers accelerate bitsandbytes datasets scipy ftfy

# 2. Clone the official Hugging Face repo
# We remove any existing folder first to ensure a clean start
!rm -rf diffusers
!git clone https://github.com/huggingface/diffusers

# 3. CRITICAL FIX: Install diffusers from the source code we just cloned
# This updates your version to 0.36.0.dev0 so the script works.
!pip install -e diffusers

# 4. Install requirements specifically for the training script
!pip install -r diffusers/examples/text_to_image/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
Cloning into 'diffusers'...
remote: Enumerating objects: 114954, done.
remote: Counting objects: 100% (453/453), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 114954 (delta 338), reused 207 (delta 199), pack-reused 114501 (from 4)
Receiving objects: 100% (114954/114954), 88.22 MiB | 29.87 MiB/s, done.
Resolving deltas: 100% (85844/85844), done.
Obtaining file:///content/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-0.editable-py3-none-any.whl size=11374 sha256=8dcc60428b7796dd58efc16cce256b128ac7411b221167e939e19423d0e

In [ ]:
# Cell: Install and Login to WandB
!pip install -q wandb

import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lionel-kamdem-kamgaing (lionel-kamdem-kamgaing-utbm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
import json
import zipfile

# --- CONFIGURATION ---
# Replace these paths with your actual folder locations
# Example: "/content/drive/MyDrive/my_images"
dataset_zip = "dataset.zip"
extract_path = "dataset"

# ---------------------
if not os.path.exists(extract_path):
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print("Unzipped!")

PATH_TO_IMAGES = "/content/dataset/dataset/images"
PATH_TO_CAPTIONS = "/content/dataset/dataset/captions"

def create_metadata(image_dir, caption_dir):
    metadata_file = os.path.join(image_dir, "metadata.jsonl")
    print(f"Generating metadata file at: {metadata_file}")

    found_count = 0
    missing_count = 0

    with open(metadata_file, 'w') as f:
        # Loop through all files in the image directory
        for filename in os.listdir(image_dir):
            if filename.lower().endswith((".jpg", ".png", ".jpeg", ".webp")):

                # 1. Determine the matching text filename
                # (Assumes image_585.jpg matches image_585.txt)
                base_name = os.path.splitext(filename)[0]
                text_filename = base_name + ".txt"
                text_path = os.path.join(caption_dir, text_filename)

                # 2. Check if the caption file exists in the OTHER folder
                if os.path.exists(text_path):
                    try:
                        with open(text_path, 'r', encoding='utf-8') as t:
                            caption_text = t.read().strip()

                        # 3. Write to JSONL
                        # We save the text CONTENT, so the file location doesn't matter anymore
                        entry = {"file_name": filename, "text": caption_text}
                        json.dump(entry, f)
                        f.write('\n')
                        found_count += 1
                    except Exception as e:
                        print(f"Error reading {text_filename}: {e}")
                else:
                    print(f"Warning: No caption found for {filename} (looked for {text_filename})")
                    missing_count += 1

    print("-" * 30)
    print(f"Done! Paired {found_count} images with captions.")
    if missing_count > 0:
        print(f"Warning: {missing_count} images were skipped because matching text files were missing.")
    print(f"You can now point your training script to: {image_dir}")

# Run the function
create_metadata(PATH_TO_IMAGES, PATH_TO_CAPTIONS)

Generating metadata file at: /content/dataset/dataset/images/metadata.jsonl
------------------------------
Done! Paired 500 images with captions.
You can now point your training script to: /content/dataset/dataset/images


In [ ]:
# Cell 3: Start Training (Updated to silence warnings)
import os

# Define paths
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
DATA_DIR = "/content/dataset/dataset/images"
OUTPUT_DIR = "/content/drive/MyDrive/Floorplan-Python"

# The Command (Added flags to fix 'accelerate' warnings)
!accelerate launch --num_processes=1 --mixed_precision="fp16" diffusers/examples/text_to_image/train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$DATA_DIR \
  --dataloader_num_workers=4 \
  --resolution=512 \
  --center_crop \
  --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=1500 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR \
  --checkpointing_steps=500 \
  --seed=1337 \
  --mixed_precision="fp16" \
  --report_to="wandb"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-12-15 15:40:56.555404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765813256.574897    4739 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765813256.580741    4739 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765813256.595514    4739 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the sa

In [ ]:
# Install the missing libraries
!pip install --upgrade diffusers transformers accelerate gradio
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline

# 1. Load the Model
print("Loading model...")
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# 2. Load Your Zellige LoRA
# Replace this path with wherever your .safetensors file is
lora_path = "/content/drive/MyDrive/Floorplan-Python/pytorch_lora_weights.safetensors"
pipe.load_lora_weights(lora_path)

# Move to GPU
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

# 3. Define the Generation Function
def generate_image(prompt):
    # We automatically add your trigger word so you don't forget it
    full_prompt = f"floorplan, {prompt}"

    image = pipe(
        full_prompt,
        num_inference_steps=30,
        guidance_scale=7.5
    ).images[0]

    return image

# 4. Create the Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Floorplan Generator")

    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(label="Describe the plan", placeholder="blue and gold star pattern...")
            generate_btn = gr.Button("Generate", variant="primary")

        with gr.Column():
            image_output = gr.Image(label="Generated Result")

    # Connect the button to the function
    generate_btn.click(fn=generate_image, inputs=prompt_input, outputs=image_output)

# 5. Launch it
print("Starting interface...")
demo.launch(share=True)

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


Loading model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

config.json: 0.00B [00:00, ?B/s]

safety_checker/model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
/tmp/ipython-input-3436067520.py:35: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Starting interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d02dafa4b84213eb8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
